In [ ]:
import SILIA
import numpy as np
from scipy.signal import square
import plotly.express as px #I like using plotly for my graphs, but feel free to use matplotlib or any other library
import plotly.graph_objects as go

Welcome to SILIA! In this tutorial, we will go over the format of the dataset for our package and how to use it to extract periodic signals from noise data. 

Let's begin by simulating a noisy dataset with two periodic signals.

First, let's generate the time axis for our signal - we can assume our instrument has a sampling rate of 2000Hz, and we want 10000 data samples. The number of samples being a power of two enhances the speed of the lock-in but is not required

We will also use the same time axis for our frequency reference, but that is not required.

In [ ]:
time = np.arange(0, 10, 1/2000) #seconds

Next, let's generate the channels for our lock-in. Since we can specify how our channels are indexed, let's enumerate our channels from 0-500 with a gap of 5 between any two channels. Specifying the channels is not required for the actual lock-in but can be convinient to use when plotting.

In [ ]:
channels = np.arange(0, 500, 5)

Now we can generate some square wave frequency references. For this tutorial, we will be locking into two signals of 75Hz and 125Hz each. Therefore, we need to generate 75Hz and 125Hz frequency references. These references are placed in a list where each reference signal is represented by a python dictionary. Each dictionary will have a key of 'time' which is associated to a list of timestamps for that reference, as well as a key of 'signal' which contains a list of the frequency reference values for each timestamp. 

For example, 
    
        references = [{'time' : [0, 1, 2, 3, 4], 'signal' : [1, 1, 1, -1, -1]}, 
            {'time' : [0, 1, 2, 3, 4], 'signal' : [1, -1, 1, -1, 1]}]
    


In [ ]:
frequencies = [75, 125] #Hz
references = []
for freq in frequencies:
    references.append({'time' : time, 'signal' : square(2 * np.pi * freq * time)})
    
#Plotting the 75Hz reference from 0 to 0.25s
fig_75 = px.line(x = references[0]['time'], y = references[0]['signal'])
fig_75.update_layout(title='75Hz Reference', xaxis_title='Time(s)',yaxis_title='Amplitude', xaxis_range = (0, 0.25))
fig_75.show()

#Plotting the 125Hz reference from 0 to 0.25s
fig_125 = px.line(x = references[1]['time'], y = references[1]['signal'])
fig_125.update_layout(title='125Hz Reference', xaxis_title='Time(s)',yaxis_title='Amplitude', xaxis_range = (0, 0.25))
fig_125.show()

Finally, we need to generate our signal input. The format of this input is a python dictionary with two keys. The first key, 'time', dictates the timestamps at which each datapoint was sampled. The second, 'signal', is a nD array where the first dimension corresponds to a time dependence. In this example, each frame of data is one dimensional so our signal is a 2D array where each row corresponds to a reading for each channel for a single timestamp.

For example, assuming two channels where one signal is $\sin{t}$ and the other is $\cos{t}$ - 

        signal_input = {'time' : [0, 1, 2, 3], 
            'signal' : [[sin(0), cos(0)],
                        [sin(1), cos(1)],
                        [sin(2), cos(2)],
                        [sin(3), cos(3)]]}

To test our lock-in, we want to add Gaussian noise to all of our channels which we will do using numpy. This noise will have a standard deviation of 2, which will be larger than our sinusoidal signal RMS of 1/$\sqrt{2}$, or signal amplitude of 1. We will add a 75Hz signal to channels 100-200, and a 125Hz signal to channels 300-400. The lock-in should be able to extract both signal amplitudes for each channel independently. 

In [ ]:
def gen_noise(standard_deviation):
    """
    Generates a random sample from a Gaussian distribution with a mean of 0 and 
    specified standard deviation.
    """
    return np.random.normal(0, standard_deviation)

signal = {'time' : time}
sig_vals = []
for t in time:
    row = []
    for channel in channels:
        if (channel >= 0 and channel < 100) or (channel >= 200 and channel < 300) or (channel >= 400 and channel < 500):
            row.append(gen_noise(2))
        elif channel >= 100 and channel < 200:
            row.append(np.sin(2 * np.pi * frequencies[0] * t) + gen_noise(2))
        elif channel >= 300 and channel < 400:
            row.append(np.sin(2 * np.pi * frequencies[1] * t) + gen_noise(2))
    sig_vals.append(row)

signal['signal'] = sig_vals
            
        

To visualize our signal, we can create a 3D plot of the sample values for each channel over a limited time window. This might take a while. We see that there is no visible signal in this mess of noise, but the lock-in has to tease it out.  

In [ ]:
fig = go.Figure(data=[go.Surface(x = signal['time'], y = channels, z= signal['signal'])])
fig.update_layout(title='Signal Input', scene = dict(xaxis_title='Time(s)',yaxis_title='Channels', 
                                                     zaxis_title = 'Signal'), xaxis_range = (0, 0.25))
fig.show()

Now that we have all our inputs, its time to lock into our desired signals! First, we have to create our lock-in amplifier. We can set and update the cutoff frequency. The cutoff frequency esentially determines how much of the Fourier basis functions you want to keep in your output. i.e. with a reference signal at 100Hz and a cutoff of 1Hz, that means the Lock-in does not filter any signal between 99 and 101Hz. 

In [ ]:
LIA = SILIA.Amplifier(1) # Create a lock-in amplifier with a cutoff frequency of 1Hz
print("The cutoff frequency is " + str(LIA.cutoff))
# Update the cutoff frequency to 0Hz (won't be exactly 0Hz)
#(i.e. effectively just the smallest possible cutoff, limited by the frequency resolution of the FFT)
LIA.update_cutoff(0) 
print("The updated cutoff frequency is " + str(LIA.cutoff))

Next, we pass in the references we want to lock into and the actual signal input into the amplify method so the software can extract the information we want. 

The output is formatted as a python dictionary with $k + 1$ keys, where $k$ is the number of frequency references passed into the program. The first key is 'references' which is a dictionary of the reference frequency and phase fit parameters which can be used for diagnostic purposes. The next $k$ keys are enumerated as 'reference {$i$}', where $i$ ranges from 1 to $k$. Each of these are associated with their own dictionary consisting of two keys, 'magnitudes' and 'phase' which lead to lists of the magnitude and phase outputs for each channel for that particular reference. 

For example, 

        out = {'ref. fit params' : {'frequencies' : [75, 125], 'phases' : [0,0]},
            'reference 1' : {'magnitudes' : [0, 0.1, 1, 1, 0.1], 'phase' : [pi, pi/4, 0, pi/2, pi/10]}
            'reference 2' : {'magnitudes' : [1, 1, 0.1, 0, 0.1], 'phase' : [pi/5, pi/3, 0, pi, pi/2]}}
            
We also included progress bars for the mixing and lowpass filtering steps of the lock-in to provide insight into how long the program is taking. the progress bar can be turned off by setting ```pbar = False``` when defining the amplifier object in the previous cell. 

In [ ]:
out = LIA.amplify(references, signal)

There is an option to interpolate (using linear interpolation) our input to ensure evenly spaced samples. This is not necessary here, but might be helpful for experimental data. 

In [ ]:
out = LIA.amplify(references, signal, interpolate = True)

Now, let's visualize our output. First, we print our fitted references rounded to the nearest integer - we expect them to be 75Hz and 125Hz. Next, let us plot the magnitudes and phase of our output for each reference by looping through all the reference keys in our output. We plot these values with respect to our channel index. We expect to see two lines on our magnitude plot, the one associated with the 75Hz reference should be 0 for all channels except channels 100-200, where it should be 1. The second line associated with the 125Hz reference should be 0 everywhere except channels 300-400, where it should be 1. 

In [ ]:
frequencies = out['ref. fit params']['frequencies']
phases = out['ref. fit params']['phases']
print("The fitted frequencies and phases for the reference signals are, \n")
for phase, freq in zip(phases, frequencies):
    print(str(round(freq)) + ' Hz, ' + str(phase) + ' rads \n')
    
x = channels

fig_magnitudes = go.Figure()
i = 0
while i < len(frequencies):
    fig_magnitudes.add_trace(go.Scatter(x= x,y = out['reference ' + str(i + 1)]['magnitudes'] ,
                             mode='lines', name = str(round(frequencies[i])) + 'Hz'))
    i += 1

fig_magnitudes.update_layout(title='Output Magnitude', xaxis_title='Channel',yaxis_title='Magnitude')
fig_magnitudes.show()

fig_phase = go.Figure()
i = 0
while i < len(frequencies):
    fig_phase.add_trace(go.Scatter(x= x,y = out['reference ' + str(i + 1)]['phases'] ,
                             mode='lines', name = str(round(frequencies[i])) + 'Hz'))
    i += 1
fig_phase.update_layout(title='Output Phase', xaxis_title='Channel',yaxis_title='Phase (rads.)')
fig_phase.show()


We get our expected output from the lock-in but can see slight error caused by the noise. In addition, we see that the phase looks like random noise but goes down closer to zero when there is a lock-in fit. 

Another benefit of doing a lock-in on a computer is the ability to get errorbars from our results. This functionality is built into SILIA. SILIA calculates output standard deviations by splitting the input data into a specified number of windows. The number of windows should range from 1 to the half of the number of full periods the input signal oscillates so each window will get at least two cycles of data. 

Adjacent windows can also overlap, to allow an increased window size while keeping the same number of windows. The overlap is governed by both the number of windows and a window size parameter in the range [len(data)/num_windows, 1]. A window size of 0.5 means each window contains half the signal data. The lower bound on the window size is required to ensure all the data is used.

For example, splitting an array of length 100 with values from 0 to 99 into 9 windows with a window size of 0.2 would result in the following windows:

$[0, 20), [10, 30), [20, 40), [30, 50), [40, 60), [50, 70), [60, 80), [70, 90), [80, 100)$

If the number of windows and window size parameters are specified, SILIA outputs additional standard deviation information using the analysis from each window. This takes longer since SILIA has to perform the original lock-in as well as perform lock-ins on each separate window of data. SILIA also outputs the indices used to splice the input as a diagnostic measure. Note: the window splitting is approximate, due to the discrete nature of the data it is not always possible to split the data exactly as specified by the user but the algorithm attempts to create a split as close to what was desired as possible. 


Adjusting our previous example, 

    out = {'ref. fit params' : {'frequencies' : [75, 125], 'phases' : [0,0]},
            'reference 1' : {'magnitudes' : [0, 0.1, 1, 1, 0.1], 'phase' : [pi, pi/4, 0, pi/2, pi/10], 
                            'magnitude stds' : [0, 0.05, 0.5, 0.5, 0.025], 'phase stds' : [pi/10, pi/8, pi/4, 0, pi/25]}
            'reference 2' : {'magnitudes' : [1, 1, 0.1, 0, 0.1], 'phase' : [pi/5, pi/3, 0, pi, pi/2], 
                            'magnitude stds' : [0, 0.5, 0.025, 0.25, 0.025], 'phase stds' : [pi/20, pi/3, pi/2, 10, pi/23]}
            'indices' : [(0, 3), (2, 5)]
           }
 
Let's run the lock-in again with errorbars. This time, splitting the data into 5 windows containing 50% of the data each. 

In [ ]:
out = LIA.amplify(references, signal, num_windows = 5, window_size = 0.5)

Let's display the indices to ensure the splitting happened properly. Note: our input array has a length of 20000 so each window should contain about 10000 samples. 

In [ ]:
out['indices']

And plotting our results with errorbars, 

In [ ]:
frequencies = out['ref. fit params']['frequencies']
phases = out['ref. fit params']['phases']
print("The fitted frequencies and phases for the reference signals are, \n")
for phase, freq in zip(phases, frequencies):
    print(str(round(freq)) + ' Hz, ' + str(phase) + 'rads \n')
    
x = channels

fig_magnitudes = go.Figure()
i = 0
while i < len(frequencies):
    fig_magnitudes.add_trace(go.Scatter(x= x,y = out['reference ' + str(i + 1)]['magnitudes'] , 
                             error_y = dict(type = 'data', array=out['reference ' + str(i + 1)]['magnitude stds'], visible = True),
                             mode='lines', name = str(round(frequencies[i])) + 'Hz'))
    i += 1

fig_magnitudes.update_layout(title='Output Magnitude', xaxis_title='Channel',yaxis_title='Magnitude')
fig_magnitudes.show()

fig_phase = go.Figure()
i = 0
while i < len(frequencies):
    fig_phase.add_trace(go.Scatter(x= x,y = out['reference ' + str(i + 1)]['phases'] ,
                             error_y = dict(type = 'data', array=out['reference ' + str(i + 1)]['phase stds'], visible = True),
                             mode='lines', name = str(round(frequencies[i])) + 'Hz'))
    i += 1
fig_phase.update_layout(title='Output Phase', xaxis_title='Channel',yaxis_title='Phase (rads.)')
fig_phase.show()

We clearly see some fluctuation in the output magnitude, but the channels are generally within error of each other. In addition, we see large fluctuations and error in the phase where there is no signal but small error when there is signal which is as expected. There is an error overestimation if there is no overlap between the windows and a significant number of windows, and vice versa if there is significant overlap between the windows and a fewer number of windows so the window parameters must be specified carefully. To learn more about the lock-in, refer to our paper on this topic, linked in the Readme.

I hope you enjoy our software!